# Typenbibliothek

pandapower verfügt über eine Bibliothek, mit Hilfe derer Eigenschaften verschiedener Kabel und Transformatoren gespeichert werden können.

Die folgenden Beispiele basieren alle auf Kabeln, sind aber übertragbar auf Transformatoren, wenn in den zugehörigen Funktionen der Parameter 'element="trafo"' gesetzt wird.


## pandapower-Standardtypen

Mit Hilfe der Funktion "available_std_types()" lassen sich die hinterlegten Daten abrufen:

In [ ]:
import pandapower as pp
net = pp.create_empty_network()
pp.available_std_types(net, element="line")

Wird ein Kabel mit Hilfe der create_line()-Funktion erzeugt, werden die Parameter aus dieser Bibliothek entnommen.

In [ ]:
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
lid = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="NAYY 4x50 SE", name="test_line")
net.line.loc[lid]

## Eigene Standardtypen definieren

Es können auch eigene Daten in der Bibliothek hinterlegt werden.

Dazu müssen mindestens die folgenden Parameter angegeben werden:

**Für Kabel**:
- r_ohm_per_km
- x_ohm_per_km
- c_nf_per_km
- max_i_ka

**Für Transformatoren:**
- sn_mva
- vn_hv_kv
- vn_lv_kv
- vk_percent
- vkr_percent
- i0_percent
- pfe_kw
- shift_degree

Um die Parameter anzulegen, müssen Sie ein dictionary nach folgendem Muster anlegen:

{"parameter1": parameter_wert1, "parameter2": paramter_wert2,...} 

Anschließend kann der neu definierte Kabeltyp mit der create_std_type()-Funktion in der Datenbank gesichert werden 

In [ ]:
net = pp.create_empty_network()
test_type = {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "max_i_ka": 0.4, "type": "cs"}
pp.create_std_type(net, name="test_type", data=test_type, element="line")
pp.available_std_types(net, element="line")

Wie man nachprüfen kann, ist der neu definierte Typ jetzt Teil der Standardbibliothek und kann wie gewohnt genutzt werden, um neue Verbindungen zu erzeugen.

In [ ]:
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
lid = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="test_type", name="test_line")
print(net.line.loc[lid])

## Lokale Standardbibliotheken

Definierte Typen werden im Netz gespeichert, wenn Sie die pp.to_json-Funktion verwenden und sind folglich auch dann verfügbar, wenn das Netz neu geladen wird.

Wenn Sie Kabeltypen lokal speichern und in mehreren Netzen wiederverwenden möchten, können Sie ein dictionary im Format

{"std_type1: {"r_ohm_per_km": r_ohm_per_km,...}, "std_type2": {"r_ohm_per_km": r_ohm_per_km,...},...}  anlegen und es mit der create_std_types-Funktion dem Netzwerk hinzufügen.

In [ ]:
net = pp.create_empty_network()
linetypes = {"typ1": {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "max_i_ka": 0.4, "type": "cs"},
             "typ2": {"r_ohm_per_km": 0.015, "x_ohm_per_km": 0.01, "c_nf_per_km": 30, "max_i_ka": 0.3, "type": "cs"}}
pp.create_std_types(net, data=linetypes, element="line")
pp.available_std_types(net)

Dieses dictionary kann nun einfach in einem lokalen Verzeichnis gesichert werden (z.B. im pickle- oder json-Format) und bei Bedarf eingelesen werden. 

Alternativ kann die gesamte Bibliothek eines existierenden Netzwerks kopiert werden.

In [ ]:
new_net = pp.create_empty_network()
pp.copy_std_types(from_net=net, to_net=new_net, element="line")
pp.available_std_types(new_net)

## Weitere Parameter hinzufügen

Es ist möglich, die Bibliothek um weitere Parameter zu ergänzen und diese in pandapower zu verwenden. Wenn Sie bspw. eine Ausfallraten-Untersuchung im Rahmen einer Verlässlichkeitsanalyse durchführen möchten, können Sie den Parameter "h_per_km" zum Standardtyp hinzufügen.


In [ ]:
import pandapower as pp
net = pp.create_empty_network()
test_type = {"r_ohm_per_km": 0.01, "x_ohm_per_km": 0.02, "c_nf_per_km": 10, "max_i_ka": 0.4, "type": "cs", "h_per_km": 0.01}
pp.create_std_type(net, name="test_type", data=test_type, element="line")
pp.available_std_types(net, element="line")

Sie können prüfen, dass der neue Parameter in der Datenbank enthalten ist.

Wir erzeugen jetzt zwei Verbindungen, eine vom neuen "test_type" und eine mit einem Typ, der den Parameter "h_per_km" nicht hat.


In [ ]:
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
l1 = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="test_type", name="test_line1")
l2 = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="NAYY 4x50 SE", name="test_line2")
net.line

Der Parameter "h_per_km" ist nicht in der Tabelle, da es sich nicht um einen Standardparameter handelt. Er kann aber über die Funktion parameter_from_std_type() geladen werden.

In [ ]:
pp.parameter_from_std_type(net, parameter="h_per_km")
net.line

Pandapower fügt den Parameter dort hinzu,wo er verfügbar ist. Die zweite Zeile enthält keinen Wert für h_per_km, weil er nicht in der Bibliothek verfügbar ist. Es ist aber alternativ möglich, einen Wert zu definieren, der angezeigt wird, wenn  der Parameter nicht verfügbar ist.


In [ ]:
pp.parameter_from_std_type(net, parameter="h_per_km", fill=0.02)
net.line

## Ändern von Standardtypen

Die change_std_type()-Function ermöglicht es, vorhandene Einträge zu ersetzen.

In [ ]:
net = pp.create_empty_network()
b1 = pp.create_bus(net, vn_kv=.4)
b2 = pp.create_bus(net, vn_kv=.4)
l2 = pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.1, std_type="NAYY 4x50 SE", name="test_line")
net.line

In [ ]:
pp.change_std_type(net, l2, "NAYY 4x150 SE", element="line")
net.line

## Typensuche


Es is möglich, nach Einträgen anhand einer Reihe von Parametern zu suchen.

In [ ]:
pp.find_std_type_by_parameter(net, data={"r_ohm_per_km": 0.1188, "x_ohm_per_km": 0.39})

Diese Parameter können zusätzlich mit einer Toleranz versehen werden, um mehrere mögliche Typen angezeigt zu bekommen.

In [ ]:
pp.find_std_type_by_parameter(net, data={"r_ohm_per_km": 0.12, "x_ohm_per_km": 0.40}, epsilon=.05)

## Lessons learned


Nach Abschluss dieses Tutorials sind Sie in der Lage, die Standardtypendatenbank:
- zu verstehen,
- zu ergänzen,
- zu überarbeiten,
- Ihren Bedürfnissen anzupassen.